In this project, I have used libraries like datetime, pandas, numpy, sklearn and xgboost

In [15]:
import numpy as np
import pandas as pd
import time
import datetime
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import GridSearchCV, cross_validate
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
import xgboost as xgb
from datetime import timedelta  
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

First, read the data from train and test and append test to train for preprocessing.
Change the date column to a unified date type.
Replace null values in reviews with mean value.

In [4]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
full = train.append( test , ignore_index = True ) 
full['purchase_date'] = pd.to_datetime(full['purchase_date'] )
full['release_date'] = pd.to_datetime(full['release_date'] )
full['purchase_date'] = full['purchase_date'].fillna(full['release_date'])+ timedelta(days=30)
full['total_positive_reviews'].fillna((full['total_positive_reviews'].mean()), inplace=True)
full['total_negative_reviews'].fillna((full['total_negative_reviews'].mean()), inplace=True)
print(full.isnull().any())
full.head(5)

categories                False
genres                    False
id                        False
is_free                   False
playtime_forever           True
price                     False
purchase_date             False
release_date              False
tags                      False
total_negative_reviews    False
total_positive_reviews    False
dtype: bool


,categories,genres,id,is_free,playtime_forever,price,purchase_date,release_date,tags,total_negative_reviews,total_positive_reviews
0,"Single-player,Steam Trading Cards,Steam Cloud","Adventure,Casual,Indie",0,False,0.000000,3700.0,2018-08-01,2013-12-10,"Indie,Adventure,Story Rich,Casual,Atmospheric,...",96.0,372.0
1,"Single-player,Partial Controller Support",RPG,1,True,0.016667,0.0,2016-12-26,2015-08-12,"Mod,Utilities,RPG,Game Development,Singleplaye...",0.0,23.0
2,"Single-player,Full controller support,Steam Tr...","Adventure,Casual,Indie",2,False,0.000000,5000.0,2018-08-01,2014-01-28,"Point & Click,Adventure,Story Rich,Comedy,Indi...",663.0,3018.0
3,"Single-player,Multi-player,Steam Achievements,...","Action,RPG",3,False,1.533333,9900.0,2016-12-28,2010-03-31,"Medieval,RPG,Open World,Strategy,Sandbox,Actio...",1746.0,63078.0
4,"Single-player,Co-op,Steam Achievements,Full co...","Action,Indie,Strategy",4,False,22.333333,4800.0,2018-04-03,2012-07-30,"Tower Defense,Co-op,Action,Strategy,Online Co-...",523.0,8841.0


Mainly take insights into month and year of purchase and release date and one hot them.

In [5]:
date_test = pd.concat((full['purchase_date'],full['release_date']),axis=1)
date_test['release_month'] = 0
date_test['release_year'] = 0
date_test['purchase_month'] = 0
date_test['purchase_year'] = 0
date_test['purchase_month'] = date_test['purchase_date'].map(lambda x: x.month)
date_test['purchase_year'] = date_test['purchase_date'].map(lambda x: x.year)
date_test['release_month'] = date_test['purchase_date'].map(lambda x: x.month)
date_test['release_year'] = date_test['purchase_date'].map(lambda x: x.year)
date_release_year = date_test['release_year']
date_release_year = pd.get_dummies(date_release_year,prefix='release_year')
date_release_year = date_release_year.drop(['release_year_2016'],axis=1)

date_release_month = date_test['release_month']
date_release_month = pd.get_dummies(date_test['release_month'],prefix='release_month')
date_release_month = date_release_month.drop(['release_month_1'],axis=1)

date_purchase_month = date_test['purchase_month']
date_purchase_month = pd.get_dummies(date_test['purchase_month'],prefix='purchase_month')
date_purchase_month = date_purchase_month.drop(['purchase_month_2'],axis=1)

date_purchase_year = date_test['purchase_year']
date_purchase_year = pd.get_dummies(date_test['purchase_year'],prefix='purchase_year')
date_purchase_year = date_purchase_year.drop(['purchase_year_2015'],axis=1)

Get the timestamp of purchase and release and use the minmax scaler to format the column for them.
Get the time interval for scaling for reviews

In [9]:
date_test['purchase_timestamp'] = date_test["purchase_date"].values.astype(np.int64) // 10 ** 9
date_test['release_timestamp'] = date_test["release_date"].values.astype(np.int64) // 10 ** 9

scaler = MinMaxScaler()
date_test[['purchase_timestamp','release_timestamp']] = scaler.fit_transform(date_test[['purchase_timestamp','release_timestamp']])
Timestamp = pd.DataFrame()
Timestamp['purchase_timestamp'] = date_test['purchase_timestamp']
Timestamp['release_timestamp'] = date_test['release_timestamp']
date = pd.concat([date_purchase_year, date_purchase_month, date_release_year,date_release_month, Timestamp],axis=1)
time_interval = date_test['release_date'].map(lambda x: 2020-x.year)

Get dummy varaibles from Genres, Categories and Tags, using .str.get_dummies function and combine three of them into tags_corr.

In [10]:
Genres = pd.DataFrame()
Genres['genres'] = full['genres']
Genres = pd.concat([Genres.drop('genres', 1), Genres['genres'].str.get_dummies(sep=",")], 1)
Categories = pd.DataFrame()
Categories['categories'] = full['categories']
Categories = pd.concat([Categories.drop('categories', 1), Categories['categories'].str.get_dummies(sep=",")], 1)
Tags = pd.DataFrame()
Tags['tags'] = full['tags']
Tags = pd.concat([Tags.drop('tags', 1), Tags['tags'].str.get_dummies(sep=",")], 1)
tags_corr = pd.concat([Genres,Categories,Tags],axis=1)

Scale the positive and negative reviews and avoid zero values in log or division

In [12]:
Total_positive_reviews = pd.DataFrame()
Total_positive_reviews['total_positive_reviews'] = np.log((full["total_positive_reviews"]+1)/time_interval)
Total_negative_reviews = pd.DataFrame()
Total_negative_reviews['total_negative_reviews'] = np.log((full["total_negative_reviews"]+1)/time_interval)
Price = pd.DataFrame()
Price["price"] = np.log(full["price"]+1)

Concat the dataframe for training

In [14]:
full_XY = pd.concat( [ Price, tags_corr,date, Total_positive_reviews , Total_negative_reviews, full['playtime_forever'] ] , axis=1 )
full_XY = full_XY.loc[:,~full_XY.columns.duplicated()]
full_X = full_XY.loc[:,'price':'total_negative_reviews']
full_train = full_XY[:357]
full_test = full_XY[357:]

train_X = full_train.loc[:,'price':'total_negative_reviews']
train_y = full_train['playtime_forever']
test_X = full_test.loc[:,'price':'total_negative_reviews']
test_y = full_test['playtime_forever']

Use xgboost and test the parameters untill we get the correct set of parameters shown below.

In [16]:
data_dmatrix = xgb.DMatrix(data=train_X,label=train_y)
X_train, X_test, y_train, y_test = train_test_split(train_X, train_y, test_size=0.2, random_state=123)

#16.4 This is the submission result
from sklearn.model_selection import GridSearchCV
cv_params = {'learning_rate': [0.095,0.098,0.1, 0.102,0.105]}
other_params = {'learning_rate': 0.1, 'n_estimators': 39, 'max_depth': 8, 'min_child_weight': 2, 'seed': 0,
                'subsample': 0.72, 'colsample_bytree': 0.78, 'gamma': 0.11, 'reg_alpha': 0.03, 'reg_lambda': 0.97}

model = xgb.XGBRegressor(**other_params)
optimized_GBM = GridSearchCV(estimator=model, param_grid=cv_params, scoring='neg_mean_squared_error', cv=5, verbose=1, n_jobs=4)
optimized_GBM.fit(train_X, train_y)
# evalute_result = optimized_GBM.grid_scores_
# print('每轮迭代运行结果:{0}'.format(evalute_result))
# print('参数的最佳取值：{0}'.format(optimized_GBM.best_params_))
# print('最佳模型得分:{0}'.format(optimized_GBM.best_score_))
print("Lowest RMSE found: ", np.sqrt(np.abs(optimized_GBM.best_score_)))
print("Best parameters found: ",optimized_GBM.best_params_)

a = optimized_GBM.predict(test_X)
a[a<0] = 0
print(len(a))
a

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  25 out of  25 | elapsed:    5.3s finished


[21:48:46] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Lowest RMSE found:  9.742091120032217
Best parameters found:  {'learning_rate': 0.1}
90


array([1.9114041e-01, 5.1005874e+00, 2.9525858e-01, 0.0000000e+00,
       5.5335937e+00, 3.0026004e-01, 1.1037109e+00, 3.0708432e-01,
       1.5925955e+00, 9.3925595e-01, 4.0547237e-01, 0.0000000e+00,
       1.4821249e+01, 5.9324324e-01, 1.1855027e+01, 1.3797539e-01,
       7.9196483e-02, 3.4473461e-01, 2.9769585e-01, 7.3555410e-01,
       7.0052373e-01, 1.4179189e+00, 8.2380950e-02, 1.8825224e-01,
       0.0000000e+00, 3.3999416e-01, 2.7229318e-01, 4.4814977e-01,
       1.3320761e+00, 2.7623296e+00, 8.2230687e-02, 1.9979580e+01,
       1.1737049e-02, 2.3031825e-01, 2.5334847e+00, 2.1999091e-02,
       3.0517325e+00, 1.7059301e+00, 1.2254424e+00, 0.0000000e+00,
       8.6613619e-01, 4.8232132e-01, 7.4355063e+00, 2.5104228e-01,
       5.2258396e-01, 1.1410849e+01, 0.0000000e+00, 4.8687470e-01,
       1.7014444e-01, 9.6109667e+00, 1.7184952e-01, 2.6830858e-01,
       2.3888052e-02, 7.9040372e-01, 2.8215439e+00, 3.2662004e-01,
       2.0398779e+00, 2.5540531e-02, 2.6591623e-01, 9.8052394e

Save the result to Xgboost.csv

In [18]:
submission = pd.DataFrame()
submission['playtime_forever'] = a

submission.index.name = 'id'

submission.to_csv('Xgboost.csv')